#### Imports

In [2]:
import pyspark
from pyspark.sql import SparkSession
from IPython.display import display
from pyspark.sql.functions import udf ,col , split, max, min, sum,year,month,dayofmonth,weekofyear,lag,avg,round,row_number
from pyspark.sql.types import DateType,StringType
from datetime import datetime
from pyspark.sql.window import Window

# BATCH PROCESSING

# Basic operations

In [3]:
spark = SparkSession.builder.appName("stocksly").getOrCreate()
spark

25/03/11 17:05:30 WARN Utils: Your hostname, ayushkhaire resolves to a loopback address: 127.0.1.1; using 192.168.107.53 instead (on interface enp1s0)
25/03/11 17:05:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 17:05:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
stocks = spark.read.csv(
    "../data_sourcing/source/archive/stocks.csv", 
    header=True, 
    inferSchema=True
)

In [5]:
stocks.show(5)

+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+
|_c0|Unnamed: 0|               low|             close|             high|             open| volume|          timestamp|stockname|
+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+
|  0|         0| 37.06499862670898| 37.42499923706055|38.06999969482422|38.02999877929688|1044400|2015-01-02 14:30:00|     WSM |
|  1|         1| 37.04999923706055|37.244998931884766|37.48500061035156|            37.25| 743200|2015-01-05 14:30:00|     WSM |
|  2|         2|             36.75| 37.06999969482422|37.45000076293945|37.21500015258789|1776000|2015-01-06 14:30:00|     WSM |
|  3|         3|37.310001373291016| 38.41999816894531|38.47999954223633|37.34999847412109|1719800|2015-01-07 14:30:00|     WSM |
|  4|         4| 38.64500045776367| 38.93000030517578|39.03499984741211|38.82500076293945|1342800

25/03/11 17:05:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, low, close, high, open, volume, timestamp, stockname
 Schema: _c0, Unnamed: 0, low, close, high, open, volume, timestamp, stockname
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/archive/stocks.csv


In [6]:
stocks.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- open: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- stockname: string (nullable = true)



In [7]:
stocks.select("stockname").show(5)

+---------+
|stockname|
+---------+
|     WSM |
|     WSM |
|     WSM |
|     WSM |
|     WSM |
+---------+
only showing top 5 rows



In [8]:
stocks.select(
    ["stockname","open", "close"]
).show(5)

+---------+-----------------+------------------+
|stockname|             open|             close|
+---------+-----------------+------------------+
|     WSM |38.02999877929688| 37.42499923706055|
|     WSM |            37.25|37.244998931884766|
|     WSM |37.21500015258789| 37.06999969482422|
|     WSM |37.34999847412109| 38.41999816894531|
|     WSM |38.82500076293945| 38.93000030517578|
+---------+-----------------+------------------+
only showing top 5 rows



In [9]:
stocks.filter(
    stocks.stockname == "TSLA "
).show(5)

25/03/11 17:05:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, low, close, high, open, volume, timestamp, stockname
 Schema: _c0, Unnamed: 0, low, close, high, open, volume, timestamp, stockname
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/archive/stocks.csv


+---+----------+------------------+------------------+------------------+------------------+--------+-------------------+---------+
|_c0|Unnamed: 0|               low|             close|              high|              open|  volume|          timestamp|stockname|
+---+----------+------------------+------------------+------------------+------------------+--------+-------------------+---------+
|  0|         0| 14.21733283996582|14.620667457580566|14.883333206176758|14.857999801635742|71466000|2015-01-02 14:30:00|    TSLA |
|  1|         1|13.810667037963867|14.005999565124512| 14.43333339691162|14.303333282470703|80527500|2015-01-05 14:30:00|    TSLA |
|  2|         2| 13.61400032043457|14.085332870483398|14.279999732971191|14.003999710083008|93928500|2015-01-06 14:30:00|    TSLA |
|  3|         3|13.985333442687988|14.063332557678224|  14.3186674118042|14.223333358764648|44526000|2015-01-07 14:30:00|    TSLA |
|  4|         4|14.000666618347168|14.041333198547363| 14.25333309173584|14.

In [10]:
stocks.filter(
    (
        (stocks.stockname == "TSLA ") |
        (stocks.stockname == "PLTR ")
    ) & (
    stocks.timestamp == "2015-01-08 14:30:00")
).show()

25/03/11 17:05:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, low, close, high, open, volume, timestamp, stockname
 Schema: _c0, Unnamed: 0, low, close, high, open, volume, timestamp, stockname
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/archive/stocks.csv


+---+----------+------------------+------------------+-----------------+------------------+--------+-------------------+---------+
|_c0|Unnamed: 0|               low|             close|             high|              open|  volume|          timestamp|stockname|
+---+----------+------------------+------------------+-----------------+------------------+--------+-------------------+---------+
|  4|         4|14.000666618347168|14.041333198547363|14.25333309173584|14.187333106994627|51637500|2015-01-08 14:30:00|    TSLA |
+---+----------+------------------+------------------+-----------------+------------------+--------+-------------------+---------+



In [11]:
stocks.filter(
    stocks.stockname.isin("TSLA ", "PLTR ") &
    (stocks.timestamp == "2015-01-08 14:30:00")
).show()

25/03/11 17:06:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, low, close, high, open, volume, timestamp, stockname
 Schema: _c0, Unnamed: 0, low, close, high, open, volume, timestamp, stockname
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/archive/stocks.csv


+---+----------+------------------+------------------+-----------------+------------------+--------+-------------------+---------+
|_c0|Unnamed: 0|               low|             close|             high|              open|  volume|          timestamp|stockname|
+---+----------+------------------+------------------+-----------------+------------------+--------+-------------------+---------+
|  4|         4|14.000666618347168|14.041333198547363|14.25333309173584|14.187333106994627|51637500|2015-01-08 14:30:00|    TSLA |
+---+----------+------------------+------------------+-----------------+------------------+--------+-------------------+---------+



In [12]:
stocks = stocks.withColumn(
    "date", split(
        col("timestamp"), 
    " ")[0]
    )
stocks = stocks.withColumn(
    "time", split(
        col("timestamp"), 
    " ")[1]
    )
stocks.show(5)

+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+----------+--------+
|_c0|Unnamed: 0|               low|             close|             high|             open| volume|          timestamp|stockname|      date|    time|
+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+----------+--------+
|  0|         0| 37.06499862670898| 37.42499923706055|38.06999969482422|38.02999877929688|1044400|2015-01-02 14:30:00|     WSM |2015-01-02|14:30:00|
|  1|         1| 37.04999923706055|37.244998931884766|37.48500061035156|            37.25| 743200|2015-01-05 14:30:00|     WSM |2015-01-05|14:30:00|
|  2|         2|             36.75| 37.06999969482422|37.45000076293945|37.21500015258789|1776000|2015-01-06 14:30:00|     WSM |2015-01-06|14:30:00|
|  3|         3|37.310001373291016| 38.41999816894531|38.47999954223633|37.34999847412109|1719800|2015-01-

25/03/11 17:06:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, low, close, high, open, volume, timestamp, stockname
 Schema: _c0, Unnamed: 0, low, close, high, open, volume, timestamp, stockname
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/archive/stocks.csv


In [13]:
date_parser = udf(
    lambda date : datetime.strptime(date, "%Y-%m-%d"),DateType()
)

In [14]:
stocks = stocks.withColumn(
    "new_date",
    date_parser(stocks.date)  
)

stocks.show(5)

25/03/11 17:06:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, low, close, high, open, volume, timestamp, stockname
 Schema: _c0, Unnamed: 0, low, close, high, open, volume, timestamp, stockname
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/archive/stocks.csv


+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+----------+--------+----------+
|_c0|Unnamed: 0|               low|             close|             high|             open| volume|          timestamp|stockname|      date|    time|  new_date|
+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+----------+--------+----------+
|  0|         0| 37.06499862670898| 37.42499923706055|38.06999969482422|38.02999877929688|1044400|2015-01-02 14:30:00|     WSM |2015-01-02|14:30:00|2015-01-02|
|  1|         1| 37.04999923706055|37.244998931884766|37.48500061035156|            37.25| 743200|2015-01-05 14:30:00|     WSM |2015-01-05|14:30:00|2015-01-05|
|  2|         2|             36.75| 37.06999969482422|37.45000076293945|37.21500015258789|1776000|2015-01-06 14:30:00|     WSM |2015-01-06|14:30:00|2015-01-06|
|  3|         3|37.310001373291016| 38.4

In [15]:
stocks.show(5)

+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+----------+--------+----------+
|_c0|Unnamed: 0|               low|             close|             high|             open| volume|          timestamp|stockname|      date|    time|  new_date|
+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+----------+--------+----------+
|  0|         0| 37.06499862670898| 37.42499923706055|38.06999969482422|38.02999877929688|1044400|2015-01-02 14:30:00|     WSM |2015-01-02|14:30:00|2015-01-02|
|  1|         1| 37.04999923706055|37.244998931884766|37.48500061035156|            37.25| 743200|2015-01-05 14:30:00|     WSM |2015-01-05|14:30:00|2015-01-05|
|  2|         2|             36.75| 37.06999969482422|37.45000076293945|37.21500015258789|1776000|2015-01-06 14:30:00|     WSM |2015-01-06|14:30:00|2015-01-06|
|  3|         3|37.310001373291016| 38.4

25/03/11 17:06:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, low, close, high, open, volume, timestamp, stockname
 Schema: _c0, Unnamed: 0, low, close, high, open, volume, timestamp, stockname
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/archive/stocks.csv


In [16]:
def upper_turner(stockname):
    return stockname.upper()

upper_turner = udf(
    upper_turner, StringType()
)

In [17]:
stocks = stocks.withColumn(
    "stockname",upper_turner(stocks.stockname)
)
stocks.show(5)

+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+----------+--------+----------+
|_c0|Unnamed: 0|               low|             close|             high|             open| volume|          timestamp|stockname|      date|    time|  new_date|
+---+----------+------------------+------------------+-----------------+-----------------+-------+-------------------+---------+----------+--------+----------+
|  0|         0| 37.06499862670898| 37.42499923706055|38.06999969482422|38.02999877929688|1044400|2015-01-02 14:30:00|     WSM |2015-01-02|14:30:00|2015-01-02|
|  1|         1| 37.04999923706055|37.244998931884766|37.48500061035156|            37.25| 743200|2015-01-05 14:30:00|     WSM |2015-01-05|14:30:00|2015-01-05|
|  2|         2|             36.75| 37.06999969482422|37.45000076293945|37.21500015258789|1776000|2015-01-06 14:30:00|     WSM |2015-01-06|14:30:00|2015-01-06|
|  3|         3|37.310001373291016| 38.4

25/03/11 17:06:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, low, close, high, open, volume, timestamp, stockname
 Schema: _c0, Unnamed: 0, low, close, high, open, volume, timestamp, stockname
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/archive/stocks.csv


In [18]:
stocks.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- open: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- stockname: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- new_date: date (nullable = true)



In [19]:
new_stocks = stocks.select(
    "stockname",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "new_date",
    "time"
)

In [20]:
new_stocks.show(5)

+---------+-----------------+-----------------+------------------+------------------+-------+----------+--------+
|stockname|             open|             high|               low|             close| volume|  new_date|    time|
+---------+-----------------+-----------------+------------------+------------------+-------+----------+--------+
|     WSM |38.02999877929688|38.06999969482422| 37.06499862670898| 37.42499923706055|1044400|2015-01-02|14:30:00|
|     WSM |            37.25|37.48500061035156| 37.04999923706055|37.244998931884766| 743200|2015-01-05|14:30:00|
|     WSM |37.21500015258789|37.45000076293945|             36.75| 37.06999969482422|1776000|2015-01-06|14:30:00|
|     WSM |37.34999847412109|38.47999954223633|37.310001373291016| 38.41999816894531|1719800|2015-01-07|14:30:00|
|     WSM |38.82500076293945|39.03499984741211| 38.64500045776367| 38.93000030517578|1342800|2015-01-08|14:30:00|
+---------+-----------------+-----------------+------------------+------------------+---

In [21]:
new_stocks.describe().show()

25/03/11 17:06:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|summary|stockname|                open|                high|                 low|               close|              volume|    time|
+-------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|  count|  4881252|             4881252|             4881252|             4881252|             4881252|             4881252| 4881252|
|   mean|     NULL|   819386.4823657873|    886035.285449156|   738057.6887514308|   806229.7104475481|2.7874788287357833E7|    NULL|
| stddev|     NULL| 9.206054466304307E7|1.0009087405909203E8| 8.228124841166525E7| 9.069951404786308E7| 8.950436302490437E8|    NULL|
|    min|       A |3.999999989900971E-6|3.999999989900971E-6|3.999999989900971E-6|3.999999989900971E-6|                   0|00:00:00|
|    max|     ZWS |     3.2500000768E10|     3.3500000256E10| 

# Basic stock analysis

In [22]:
new_stocks.groupBy("stockname").max("Open").show(15)

+---------+------------------+
|stockname|         max(Open)|
+---------+------------------+
|     ROP | 585.2000122070312|
|     WSM | 218.4499969482422|
|      GS | 669.0800170898438|
|     MAN | 135.4600067138672|
|    GSHD |177.00999450683594|
|    GKOS |160.39999389648438|
|     BHP | 72.72078704833984|
|     PFS |28.950000762939453|
|      ZM |             572.5|
|     SUZ |14.140000343322754|
|    GATX |            167.25|
|    ENOV | 93.16695404052734|
|     AFL |115.44000244140624|
|     NEM | 85.55000305175781|
|     FNF | 63.77999877929688|
+---------+------------------+
only showing top 15 rows



In [23]:
new_stocks.groupBy("stockname").max("Open").withColumnRenamed("max(Open)", "MaxStockPrice").show(15)

+---------+------------------+
|stockname|     MaxStockPrice|
+---------+------------------+
|     ROP | 585.2000122070312|
|     WSM | 218.4499969482422|
|      GS | 669.0800170898438|
|     MAN | 135.4600067138672|
|    GSHD |177.00999450683594|
|    GKOS |160.39999389648438|
|     BHP | 72.72078704833984|
|     PFS |28.950000762939453|
|      ZM |             572.5|
|     SUZ |14.140000343322754|
|    GATX |            167.25|
|    ENOV | 93.16695404052734|
|     AFL |115.44000244140624|
|     NEM | 85.55000305175781|
|     FNF | 63.77999877929688|
+---------+------------------+
only showing top 15 rows



In [24]:
new_stocks.groupBy("stockname").agg(max("Open").alias("MaxStockPrice")).show(15)

+---------+------------------+
|stockname|     MaxStockPrice|
+---------+------------------+
|     ROP | 585.2000122070312|
|     WSM | 218.4499969482422|
|      GS | 669.0800170898438|
|     MAN | 135.4600067138672|
|    GSHD |177.00999450683594|
|    GKOS |160.39999389648438|
|     BHP | 72.72078704833984|
|     PFS |28.950000762939453|
|      ZM |             572.5|
|     SUZ |14.140000343322754|
|    GATX |            167.25|
|    ENOV | 93.16695404052734|
|     AFL |115.44000244140624|
|     NEM | 85.55000305175781|
|     FNF | 63.77999877929688|
+---------+------------------+
only showing top 15 rows



In [25]:
new_stocks.groupBy("stockname").agg(
    max("open").alias("MaxStockPrice"),
    sum("volume").alias("TotalVolume")
).show(15)

+---------+------------------+-----------+
|stockname|     MaxStockPrice|TotalVolume|
+---------+------------------+-----------+
|     ROP | 585.2000122070312| 1265775000|
|     WSM | 218.4499969482422| 6790355900|
|      GS | 669.0800170898438| 7454028500|
|     MAN | 135.4600067138672| 1408781400|
|    GSHD |177.00999450683594|  384962900|
|    GKOS |160.39999389648438| 1251221700|
|     BHP | 72.72078704833984| 8068724943|
|     PFS |28.950000762939453|  850348800|
|      ZM |             572.5| 7172002600|
|     SUZ |14.140000343322754| 1610642200|
|    GATX |            167.25|  642718300|
|    ENOV | 93.16695404052734| 1635139281|
|     AFL |115.44000244140624| 8501723600|
|     NEM | 85.55000305175781|20342713000|
|     FNF | 63.77999877929688| 4393819967|
+---------+------------------+-----------+
only showing top 15 rows



In [26]:
new_stocks = (
    new_stocks.withColumn("Year", year(new_stocks.new_date))
    .withColumn("Month", month(new_stocks.new_date))
    .withColumn("Day", dayofmonth(new_stocks.new_date))
    .withColumn("Week", weekofyear(new_stocks.new_date))
)

In [27]:
new_stocks.show(5)

+---------+-----------------+-----------------+------------------+------------------+-------+----------+--------+----+-----+---+----+
|stockname|             open|             high|               low|             close| volume|  new_date|    time|Year|Month|Day|Week|
+---------+-----------------+-----------------+------------------+------------------+-------+----------+--------+----+-----+---+----+
|     WSM |38.02999877929688|38.06999969482422| 37.06499862670898| 37.42499923706055|1044400|2015-01-02|14:30:00|2015|    1|  2|   1|
|     WSM |            37.25|37.48500061035156| 37.04999923706055|37.244998931884766| 743200|2015-01-05|14:30:00|2015|    1|  5|   2|
|     WSM |37.21500015258789|37.45000076293945|             36.75| 37.06999969482422|1776000|2015-01-06|14:30:00|2015|    1|  6|   2|
|     WSM |37.34999847412109|38.47999954223633|37.310001373291016| 38.41999816894531|1719800|2015-01-07|14:30:00|2015|    1|  7|   2|
|     WSM |38.82500076293945|39.03499984741211| 38.64500045776

In [28]:
yearly = new_stocks.groupBy(
    ['stockname', 'Year']
).agg(
    max("open").alias("YearlyHigh"), 
    min("open").alias("YearlyLow")
)
yearly.show(5)

+---------+----+------------------+------------------+
|stockname|Year|        YearlyHigh|         YearlyLow|
+---------+----+------------------+------------------+
|     WSM |2021|110.99500274658205|50.790000915527344|
|     ADP |2020| 181.3600006103516|108.52999877929688|
|    KLIC |2023|  59.7599983215332| 41.33000183105469|
|    CROX |2015|  15.9399995803833| 9.329999923706056|
|     CNC |2021| 84.95999908447266|              57.5|
+---------+----+------------------+------------------+
only showing top 5 rows



In [29]:
monthly = new_stocks.groupBy(
    ["stockname", "year", "month"]  # Ensure lowercase if needed
).agg(
    max("Open").alias("MonthlyHigh"), 
    min("Open").alias("MonthlyLow")
)
monthly.show(5)

+---------+----+-----+-----------------+------------------+
|stockname|year|month|      MonthlyHigh|        MonthlyLow|
+---------+----+-----+-----------------+------------------+
|    GMED |2022|   11|72.05000305175781|61.790000915527344|
|     ALK |2015|    8|81.55999755859375| 70.02999877929688|
|     ALK |2021|    9|61.63999938964844|56.130001068115234|
|     ADP |2016|    6|89.73999786376953| 85.94000244140625|
|     PBH |2019|    4|30.11000061035156|27.739999771118164|
+---------+----+-----+-----------------+------------------+
only showing top 5 rows



In [30]:
weekly = new_stocks.groupBy(
    ['stockname', 'Year', 'Week']
).agg(
    max("Open").alias("WeekHigh"), min("Open").alias("WeekLow")
)
weekly.show(5)

+---------+----+----+------------------+------------------+
|stockname|Year|Week|          WeekHigh|           WeekLow|
+---------+----+----+------------------+------------------+
|     WSM |2019|  17|28.700000762939453|27.864999771118164|
|     WSM |2020|  40|45.900001525878906| 43.92499923706055|
|     PNR |2017|  40|47.011417388916016|    45.60107421875|
|     PNR |2019|  35|  36.2599983215332|34.630001068115234|
|     PNR |2021|  22| 70.70999908447266|  68.0199966430664|
+---------+----+----+------------------+------------------+
only showing top 5 rows



In [31]:
weekly = weekly.withColumn(
    "Spread", weekly['WeekHigh'] - weekly['WeekLow']
)
weekly.show(5)

+---------+----+----+------------------+------------------+------------------+
|stockname|Year|Week|          WeekHigh|           WeekLow|            Spread|
+---------+----+----+------------------+------------------+------------------+
|     WSM |2019|  17|28.700000762939453|27.864999771118164|0.8350009918212891|
|     WSM |2020|  40|45.900001525878906| 43.92499923706055|1.9750022888183594|
|     PNR |2017|  40|47.011417388916016|    45.60107421875|1.4103431701660156|
|     PNR |2019|  35|  36.2599983215332|34.630001068115234|1.6299972534179688|
|     PNR |2021|  22| 70.70999908447266|  68.0199966430664|  2.69000244140625|
+---------+----+----+------------------+------------------+------------------+
only showing top 5 rows



# Joins

In [32]:
historic_stocks = new_stocks.join(
    yearly, 
    (
        new_stocks.stockname==yearly.stockname
    ) & (
        new_stocks.Year == yearly.Year
        ),
    'inner'
).drop(
    yearly.Year, 
    yearly.stockname
)

In [33]:
historic_stocks.show(5)

+------------------+------------------+------------------+------------------+-------+----------+--------+-----+---+----+---------+----+------------------+------------------+
|              open|              high|               low|             close| volume|  new_date|    time|Month|Day|Week|stockname|Year|        YearlyHigh|         YearlyLow|
+------------------+------------------+------------------+------------------+-------+----------+--------+-----+---+----+---------+----+------------------+------------------+
| 50.90499877929688| 52.90499877929688|            49.375| 51.47499847412109|2644400|2021-01-04|14:30:00|    1|  4|   1|     WSM |2021|110.99500274658205|50.790000915527344|
|50.790000915527344| 52.47499847412109| 50.72499847412109| 52.08000183105469|1747800|2021-01-05|14:30:00|    1|  5|   1|     WSM |2021|110.99500274658205|50.790000915527344|
|              51.5|53.935001373291016|              51.5| 53.45500183105469|2268800|2021-01-06|14:30:00|    1|  6|   1|     WSM |

In [34]:
cond = [
    (
        historic_stocks.stockname==weekly.stockname
    ) & (
        historic_stocks.Year == weekly.Year
    ) & (
        historic_stocks.Week == weekly.Week
    )
]
historic_stocks = historic_stocks.join(
    weekly, 
    cond, 
    'inner'
).drop(
    weekly.Year, 
    historic_stocks.stockname, 
    weekly.Week
)

In [35]:
historic_stocks.show(5)

+------------------+------------------+------------------+-----------------+-------+----------+--------+-----+---+----+-----------------+-----------------+---------+----+--------+-----------------+------------------+
|              open|              high|               low|            close| volume|  new_date|    time|Month|Day|Year|       YearlyHigh|        YearlyLow|stockname|Week|WeekHigh|          WeekLow|            Spread|
+------------------+------------------+------------------+-----------------+-------+----------+--------+-----+---+----+-----------------+-----------------+---------+----+--------+-----------------+------------------+
| 40.31999969482422|40.459999084472656| 39.70000076293945|39.79999923706055|2041800|2015-01-05|14:30:00|    1|  5|2015|43.59000015258789|33.18000030517578|       A |   2|    41.0|39.52000045776367|1.4799995422363281|
|39.810001373291016| 40.02000045776367| 39.02000045776367|39.18000030517578|2080600|2015-01-06|14:30:00|    1|  6|2015|43.5900001525

In [36]:
historic_stocks.columns

['open',
 'high',
 'low',
 'close',
 'volume',
 'new_date',
 'time',
 'Month',
 'Day',
 'Year',
 'YearlyHigh',
 'YearlyLow',
 'stockname',
 'Week',
 'WeekHigh',
 'WeekLow',
 'Spread']

In [37]:
historic_stocks = historic_stocks.join(
    monthly, [
        'stockname', 
        'Year', 
        'Month'
    ]
)
historic_stocks.show()

+---------+----+-----+------------------+------------------+------------------+------------------+-------+----------+--------+---+-----------------+-----------------+----+------------------+------------------+------------------+-----------------+------------------+
|stockname|Year|Month|              open|              high|               low|             close| volume|  new_date|    time|Day|       YearlyHigh|        YearlyLow|Week|          WeekHigh|           WeekLow|            Spread|      MonthlyHigh|        MonthlyLow|
+---------+----+-----+------------------+------------------+------------------+------------------+-------+----------+--------+---+-----------------+-----------------+----+------------------+------------------+------------------+-----------------+------------------+
|       A |2015|    2| 39.22999954223633| 39.43000030517578| 38.95000076293945|39.040000915527344|3586100|2015-02-09|14:30:00|  9|43.59000015258789|33.18000030517578|   7|39.970001220703125| 39.22999954

In [38]:
final_stocks = historic_stocks.select(
    [
        'stockname', 
        'Year',
        'Month', 
        'Day', 
        'Week', 
        'new_date',
        'Volume',
        'Open',
        'Low',
        'High',
        'Close', 
        'YearlyHigh',
        'YearlyLow',
        'WeekHigh',
        'WeekLow',
        'MonthlyHigh',
        'MonthlyLow'
    ]
)

# advanced analysis

In [39]:
final_stocks.select('stockname').distinct().collect()

25/03/11 17:25:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/11 17:25:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/11 17:25:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


[Row(stockname='IRM '),
 Row(stockname='ROP '),
 Row(stockname='SFM '),
 Row(stockname='GRAB '),
 Row(stockname='UNM '),
 Row(stockname='ACHR '),
 Row(stockname='DG '),
 Row(stockname='VTMX '),
 Row(stockname='CWEN '),
 Row(stockname='JNPR '),
 Row(stockname='OVV '),
 Row(stockname='VMI '),
 Row(stockname='LBRT '),
 Row(stockname='CPB '),
 Row(stockname='DVA '),
 Row(stockname='LEN-B '),
 Row(stockname='PSTG '),
 Row(stockname='XEL '),
 Row(stockname='OMAB '),
 Row(stockname='GFS '),
 Row(stockname='MTH '),
 Row(stockname='CHFFF '),
 Row(stockname='WSM '),
 Row(stockname='GS '),
 Row(stockname='RMD '),
 Row(stockname='RXO '),
 Row(stockname='MAN '),
 Row(stockname='EXPE '),
 Row(stockname='GSHD '),
 Row(stockname='SRPT '),
 Row(stockname='VRRM '),
 Row(stockname='FRT '),
 Row(stockname='PNC '),
 Row(stockname='TDS '),
 Row(stockname='CPA '),
 Row(stockname='BTSG '),
 Row(stockname='ITGR '),
 Row(stockname='NTES '),
 Row(stockname='GLNG '),
 Row(stockname='SOLV '),
 Row(stockname='ODD '

In [40]:
final_stocks.show(5)

+---------+----+-----+---+----+----------+-------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+
|stockname|Year|Month|Day|Week|  new_date| Volume|              Open|              Low|              High|             Close|       YearlyHigh|        YearlyLow|          WeekHigh|          WeekLow|      MonthlyHigh|        MonthlyLow|
+---------+----+-----+---+----+----------+-------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+
|       A |2015|    2|  9|   7|2015-02-09|3586100| 39.22999954223633|38.95000076293945| 39.43000030517578|39.040000915527344|43.59000015258789|33.18000030517578|39.970001220703125|39.22999954223633|42.34000015258789|37.900001525878906|
|       A |2015|    2| 10|   7|2015-02-10|1408600|39.310

In [41]:
final_stocks.createOrReplaceTempView('stockData')

In [42]:
spark.sql("SELECT * FROM stockData where stockname='MSFT ' and Year=2015").show(5)

+---------+----+-----+---+----+----------+--------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+
|stockname|Year|Month|Day|Week|  new_date|  Volume|              Open|               Low|             High|            Close|        YearlyHigh|        YearlyLow|          WeekHigh|           WeekLow|       MonthlyHigh|        MonthlyLow|
+---------+----+-----+---+----+----------+--------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+
|    MSFT |2015|    6|  1|  23|2015-06-01|28837300|47.060001373291016|46.619998931884766|47.77000045776367|47.22999954223633|56.470001220703125|40.34000015258789|47.369998931884766|46.310001373291016|47.369998931884766|44.709999084472656|
|    MSFT |2015|    6|  2|  23|2015-06-02|21

In [43]:
snapshot_open = historic_stocks.select(['new_date','stockname','open'])
snapshot_open.show()

+----------+---------+------------------+
|  new_date|stockname|              open|
+----------+---------+------------------+
|2015-01-05|       A | 40.31999969482422|
|2015-01-06|       A |39.810001373291016|
|2015-01-07|       A | 39.52000045776367|
|2015-01-08|       A |  40.2400016784668|
|2015-01-09|       A |              41.0|
|2015-01-20|       A | 38.43000030517578|
|2015-01-21|       A |             37.75|
|2015-01-22|       A |  38.5099983215332|
|2015-01-23|       A | 39.59999847412109|
|2015-03-02|       A | 42.34000015258789|
|2015-03-03|       A | 42.45000076293945|
|2015-03-04|       A | 41.95000076293945|
|2015-03-05|       A |  41.9900016784668|
|2015-03-06|       A | 41.95000076293945|
|2015-04-20|       A | 43.33000183105469|
|2015-04-21|       A | 43.41999816894531|
|2015-04-22|       A |42.720001220703125|
|2015-04-23|       A | 42.41999816894531|
|2015-04-24|       A |42.709999084472656|
|2015-05-18|       A | 42.04999923706055|
+----------+---------+------------

In [44]:
lag1day = Window.partitionBy("stockname").\
    orderBy('new_date')

In [45]:
snapshot_open.withColumn(
    "previous_open",
    lag(
        'open',1
    ).over(
        lag1day
    )
).show()

+----------+---------+------------------+------------------+
|  new_date|stockname|              open|     previous_open|
+----------+---------+------------------+------------------+
|2015-01-02|   AAGIY |22.459999084472656|              NULL|
|2015-01-05|   AAGIY | 22.09000015258789|22.459999084472656|
|2015-01-06|   AAGIY |21.479999542236328| 22.09000015258789|
|2015-01-07|   AAGIY | 21.63999938964844|21.479999542236328|
|2015-01-08|   AAGIY |21.940000534057617| 21.63999938964844|
|2015-01-09|   AAGIY | 22.11000061035156|21.940000534057617|
|2015-01-12|   AAGIY |22.040000915527344| 22.11000061035156|
|2015-01-13|   AAGIY | 22.65999984741211|22.040000915527344|
|2015-01-14|   AAGIY | 22.63999938964844| 22.65999984741211|
|2015-01-15|   AAGIY |23.270000457763672| 22.63999938964844|
|2015-01-16|   AAGIY |23.190000534057617|23.270000457763672|
|2015-01-20|   AAGIY | 23.21999931335449|23.190000534057617|
|2015-01-21|   AAGIY |23.030000686645508| 23.21999931335449|
|2015-01-22|   AAGIY |23

In [46]:
movingaverage = Window.partitionBy("stockname").orderBy(
    "new_date"
).rowsBetween(-50,0)

In [47]:
(
    snapshot_open.withColumn(
        "MA50", avg('open').over(movingaverage)
    ).withColumn(
        "MA50", round("MA50", 2) 
    )
).show()

+----------+---------+------------------+-----+
|  new_date|stockname|              open| MA50|
+----------+---------+------------------+-----+
|2015-01-02|   AAGIY |22.459999084472656|22.46|
|2015-01-05|   AAGIY | 22.09000015258789|22.27|
|2015-01-06|   AAGIY |21.479999542236328|22.01|
|2015-01-07|   AAGIY | 21.63999938964844|21.92|
|2015-01-08|   AAGIY |21.940000534057617|21.92|
|2015-01-09|   AAGIY | 22.11000061035156|21.95|
|2015-01-12|   AAGIY |22.040000915527344|21.97|
|2015-01-13|   AAGIY | 22.65999984741211|22.05|
|2015-01-14|   AAGIY | 22.63999938964844|22.12|
|2015-01-15|   AAGIY |23.270000457763672|22.23|
|2015-01-16|   AAGIY |23.190000534057617|22.32|
|2015-01-20|   AAGIY | 23.21999931335449|22.39|
|2015-01-21|   AAGIY |23.030000686645508|22.44|
|2015-01-22|   AAGIY |23.149999618530277|22.49|
|2015-01-23|   AAGIY |23.149999618530277|22.54|
|2015-01-26|   AAGIY |23.239999771118164|22.58|
|2015-01-27|   AAGIY |23.270000457763672|22.62|
|2015-01-28|   AAGIY |23.850000381469727

In [48]:
maximumstock = Window.partitionBy(
    "stockname"
).orderBy(snapshot_open.open.desc())

In [49]:
snapshot_open.withColumn(
    "maxopen",
    row_number().over(maximumstock)
).show()

+----------+---------+------------------+-------+
|  new_date|stockname|              open|maxopen|
+----------+---------+------------------+-------+
|2021-01-19|   AAGIY |55.650001525878906|      1|
|2021-02-17|   AAGIY |55.310001373291016|      2|
|2021-01-20|   AAGIY |54.880001068115234|      3|
|2021-02-23|   AAGIY |              54.5|      4|
|2021-01-15|   AAGIY | 54.34000015258789|      5|
|2021-02-16|   AAGIY | 54.16999816894531|      6|
|2021-04-29|   AAGIY | 54.16999816894531|      7|
|2021-05-18|   AAGIY | 54.06999969482422|      8|
|2021-01-12|   AAGIY | 53.68999862670898|      9|
|2021-02-09|   AAGIY | 53.52000045776367|     10|
|2021-05-28|   AAGIY | 53.40999984741211|     11|
|2021-05-17|   AAGIY | 53.31999969482422|     12|
|2021-02-10|   AAGIY |  53.2599983215332|     13|
|2021-05-14|   AAGIY |  53.2599983215332|     14|
|2021-05-07|   AAGIY |53.220001220703125|     15|
|2021-01-13|   AAGIY |53.209999084472656|     16|
|2021-02-12|   AAGIY | 53.20000076293945|     17|


In [50]:
snapshot_open.withColumn(
    "maxopen",
    row_number().over(maximumstock)
).filter('maxopen=5').show()

+----------+---------+------------------+-------+
|  new_date|stockname|              open|maxopen|
+----------+---------+------------------+-------+
|2021-09-13|       A |177.66000366210938|      5|
|2022-03-31|      AA | 91.66999816894533|      5|
|2021-01-15|   AAGIY | 54.34000015258789|      5|
|2018-01-22|     AAL |  57.9900016784668|      5|
|2024-12-09|    AAON |            138.25|      5|
|2022-01-07|     AAP | 239.9600067138672|      5|
|2024-12-31|    AAPL |252.44000244140625|      5|
|2021-11-04|      AB | 56.45000076293945|      5|
|2024-09-27|   ABBNY |58.900001525878906|      5|
|2024-11-04|    ABBV | 203.7899932861328|      5|
|2024-11-11|    ABCB | 71.19000244140625|      5|
|2018-03-26|    ABEV | 7.239999771118164|      5|
|2025-02-18|     ABG | 302.4599914550781|      5|
|2024-11-13|     ABM |             57.75|      5|
|2021-03-16|    ABNB | 209.1100006103516|      5|
|2021-10-28|     ABR | 19.88999938964844|      5|
|2021-12-27|     ABT |139.82000732421875|      5|


# save data

In [51]:
final_stocks.write.option(
    "header",True
).partitionBy(
    "stockname"
).mode("overwrite").parquet(
    "output/final_parquet"
)

In [52]:
final_stocks.write.option(
    "header",True
).partitionBy(
    "stockname"
).mode("overwrite").csv(
    "output/final_csv"
)

In [53]:
! ls output/

final_csv  final_parquet


# end